In [1]:
import os

%pwd

'd:\\Deep_Learning_Projects\\Sports_Classification\\notebooks'

In [2]:
os.chdir("../")
%pwd

'd:\\Deep_Learning_Projects\\Sports_Classification'

In [3]:
import tensorflow as tf

In [4]:
model = tf.keras.models.load_model(
    r"D:\Deep_Learning_Projects\Sports_Classification\artifacts\model_training\model.h5"
)

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    model_path: Path
    training_data_path: Path
    all_params: dict
    mlflow_url: str
    image_size: list
    batch_size: int

In [6]:
MLFLOW_TRACKING_URI="https://dagshub.com/Nishchal30/Sports_Classification.mlflow" 
MLFLOW_TRACKING_USERNAME="Nishchal30" 
MLFLOW_TRACKING_PASSWORD="9f254bffb4746366bc64fdd80421c69784022242" 
# python script.py

In [7]:
from src.Sports_Classification.Constants import *
from src.Sports_Classification.utils.utils import *


class ConfigurationManager:
    def __init__(
        self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH
    ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config["artifacts_root"]])

    def get_evaluation_config(self) -> ModelEvaluationConfig:
        eval_config = ModelEvaluationConfig(
            model_path =r"D:\Deep_Learning_Projects\Sports_Classification\artifacts\model_training\model.h5",
            training_data_path=r"D:\Deep_Learning_Projects\Sports_Classification\artifacts\data_ingestion\data\train",
            all_params=self.params,
            mlflow_url="https://dagshub.com/Nishchal30/Sports_Classification.mlflow",
            image_size=self.params["IMAGE_SIZE"],
            batch_size=self.params["BATCH_SIZE"],
        )

        return eval_config

In [8]:
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [16]:
class Model_Evaluation:

    def __init__(self, config : ModelEvaluationConfig):
        self.config = config

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1/255,
            validation_split = 0.2
        )

        dataflow_kwargs = dict(
            target_size = self.config.image_size[:-1],
            batch_size = self.config.batch_size,
            interpolation = "bilinear"
        )


        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )


        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data_path,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(model_path : Path) -> tf.keras.Model:
        return tf.keras.models.load_model(model_path)
    

    def eveluate_model(self):
        self.model = self.load_model(self.config.model_path)
        self.train_valid_generator()
        self.score = self.model.evaluate(self.valid_generator)

    def save_score(self):
        scores = {"loss" : self.score[0], "accuracy" : self.score[1]}
        save_json(path = Path("model_score.json"), data = scores)


    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_url)
        print(self.config.mlflow_url)
        tracking_url = urlparse(mlflow.get_tracking_uri()).scheme
        print(tracking_url)

        with mlflow.start_run() as run:
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"loss" : self.score[0], "accuracy" : self.score[1]})

            
            if tracking_url != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16")
            else:
                mlflow.keras.log_model(self.model,"model")

In [17]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Model_Evaluation(eval_config)
    evaluation.eveluate_model()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

Found 77 images belonging to 80 classes.
5/5 [==============================] - 15s 3s/step - loss: 9.3620 - accuracy: 0.0779


2024/03/03 12:00:45 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


https://dagshub.com/Nishchal30/Sports_Classification.mlflow
file
INFO:tensorflow:Assets written to: C:\Users\admin\AppData\Local\Temp\tmpd61kjml3\model\data\model\assets
